## Create Data files of ellipse metrics preceding anomalous temperatures.

This file is changed because I figured out how to actually create independent samples. 

In [17]:
#relevant import statements
import numpy as np
import math
import xarray as xr 
import pickle 
import matplotlib.pyplot as plt

from leadtime import leadtime_3classif
from leadtime import leadtime_2classif

In [18]:
infile = open("./relevant_diags_NaNfree/1959wind10_NaN.p", 'rb') 
wind = pickle.load(infile)
infile.close()

infile = open("./relevant_diags_NaNfree/1959size10_NaN.p", 'rb') 
size = pickle.load(infile)
infile.close()

infile = open("./relevant_diags_NaNfree/1959cenlat10_NaN.p", 'rb')
cenlat = pickle.load(infile)
infile.close()

#eddy-heat flux anom MDTF at 100hPa 
infile = open("./relevant_diags_NaNfree/1959ehf100_NaN.p", 'rb') 
ehf100 = pickle.load(infile)
infile.close()

#NAO Index
infile = open("./relevant_diags_NaNfree/1959nao_NaN.p", 'rb') 
nao = pickle.load(infile)
infile.close()

In [19]:
infile = open("./Updated_Diags/number_1959.p", 'rb') 
classif = pickle.load(infile)
infile.close()

classif = np.delete(classif,[20],0)

In [20]:
indices = np.isnan(wind)

for val in indices:
    if np.any(val) != False:
        print("True")

In [21]:
#upload temperature region file 
infile = open("../../CNN_Folders/DataProcess/1959weightedANOMwnc_temp1000.p", 'rb')
temp = pickle.load(infile)
infile.close()

In [22]:
temp = np.delete(temp ,[120],1)

In [23]:
temp.shape

(63, 151)

In [24]:
classif.shape

(63, 151)

In [25]:
##import climo probabilities 
infile = open("../../CNN_Folders/1959wnc_3climoprob.p", 'rb')
temp_prob = pickle.load(infile)
infile.close()

In [26]:
temp_prob

array([[0.33333333, 0.3968254 , 0.26984127],
       [0.26984127, 0.49206349, 0.23809524],
       [0.23809524, 0.57142857, 0.19047619],
       ...,
       [0.23809524, 0.46031746, 0.3015873 ],
       [0.26984127, 0.46031746, 0.26984127],
       [0.26984127, 0.31746032, 0.41269841]])

In [27]:
climo = temp_prob.reshape(63,152,3)
climo = np.delete(climo,[120],1)
climo.shape

(63, 151, 3)

In [28]:
t_c = temp

In [29]:
##designate quartiles of values 
c_mx = np.nanquantile(t_c,1) #maximum
c_q70 = np.nanquantile(t_c,0.7) #seventy
c_q30 = np.nanquantile(t_c,0.3)
c_mn = np.nanquantile(t_c,0) #minimum"

In [30]:
###Paste appropriate lead time cell below

In [32]:
##tc = index 0, clim = index 1
#range_low = [10,15,20,24,30]
#range_high = 151
#leadtop = [5,10,15,19,25]
#leadbott = [0,5,10,14,20]

range_low = [20,24,30]
range_high = 151
leadtop = [15,19,25]
leadbott = [10,14,20]

for i in range(len(leadtop)):
    #set empty list of metrics
    w = []
    sn = []
    ct = []
    eh = []
    cs = []

    ##empty list for temperature binary and climo probs
    tc = []
    clim = []
    
    ##run leadtime definition
    lead = leadtime_3classif(t_c,range_low[i],range_high,leadtop[i],leadbott[i],c_q30, c_q70, c_mx, 
                             w,sn,ct,eh,cs,tc,clim,wind,size,cenlat,nao,classif,climo)

    print("checking sizes of tc and wind ...")
    print(len(tc))
    print(len(w))

    ##stack metrics
    w1 = np.vstack(w)
    sn1 = np.vstack(sn)
    ct1 = np.vstack(ct)
    eh1 = np.vstack(eh)
    cs1 = np.vstack(cs)

    print(w1.shape)

    test = np.empty((len(w),5,5))
    test[:,:,0]= w1
    test[:,:,1]= sn1
    test[:,:,2]= ct1
    test[:,:,3]= eh1
    test[:,:,4]= cs1

    tc1 = np.stack(tc)
    clim1 = np.stack(clim)
    print(clim1.shape)

    pickle.dump(tc1, open("./redo_NNfiles/WNC_temp3class_"+str(leadbott[i])+".p", 'wb'))
    pickle.dump(clim1, open("./redo_NNfiles/WNC_climo3class_"+str(leadbott[i])+".p", 'wb'))
    pickle.dump(test, open("./redo_NNfiles/WNC_DataAnom_3class_"+str(leadbott[i])+".p", 'wb'))

checking sizes of tc and wind ...
2762
2762
(2762, 5)
(2762, 3)
checking sizes of tc and wind ...
2819
2819
(2819, 5)
(2819, 3)
checking sizes of tc and wind ...
2734
2734
(2734, 5)
(2734, 3)
